
# Building a Multi-Agent System with MCP

*Adding controls to the MAS and MCP system*

copyright 2025, Denis Rothman

**Making the Agent System More Robust**

We will now add several important controls to our multi agent system. These additions will change our functional prototype into a more reliable and intelligent system. We will focus on handling real world failures ensuring data is correct and implementing a quality control process.

First we will make our communication with the API more resilient. We will replace the original call_llm function with a new one called `call_llm_robust`. This new function adds an automatic retry mechanism. The system will now attempt to call the API multiple times if it fails. This protects our system from temporary network problems.

Next we will validate all messages. A new function called `validate_mcp_message` will check every message passed between agents. It will confirm that the message is formatted correctly and contains all the required information. This prevents corrupted data from breaking our workflow.

We will also expand our agent team with a new `validator_agent`. This agent has one job. It acts as a fact checker. It compares the writer's draft against the researcher's summary to make sure the facts are consistent. This adds a layer of quality control to the system.

Finally we will create a self correcting workflow. We will upgrade the orchestrator to include a validation and revision loop. If the validator agent finds a problem with the draft the new logic will send the content back to the writer agent with feedback. The writer will then make a revision. The system can now correct its own mistakes to produce a more accurate final output.

In [1]:
import json
import time
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

#@title 1. Initializing the Gemini Client with .env
def initialize_gemini_client():
    """Initialize Gemini client with API key from environment."""
    api_key = os.getenv('GOOGLE_API_KEY')
    
    if not api_key:
        raise ValueError(
            "GOOGLE_API_KEY not found in environment variables. "
            "Please check your .env file."
        )
    
    client = genai.Client(api_key=api_key)
    print("✓ Gemini client initialized successfully.")
    return client

# Initialize the client
client = initialize_gemini_client()

✓ Gemini client initialized successfully.


In [3]:
#@title 2.Defining the Protocol: The MCP Standard
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "1.0",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

print("--- Example MCP Message (Our Simplified Version) ---")
example_mcp = create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of the Mediterranean diet.",
    metadata={"task_id": "T-123", "priority": "high"}
)
print(json.dumps(example_mcp, indent=2))

--- Example MCP Message (Our Simplified Version) ---
{
  "protocol_version": "1.0",
  "sender": "Orchestrator",
  "content": "Research the benefits of the Mediterranean diet.",
  "metadata": {
    "task_id": "T-123",
    "priority": "high"
  }
}


# 3.New: Robust Component Controls
This is a completely new section that introduces the core engineering principles of resilience and reliability. We've added two crucial functions here. The call_llm_robust function hardens our system against network failures by adding an automatic retry mechanism. The validate_mcp_message function acts as a critical guardrail, ensuring that every message passed between our agents is correctly formatted, which prevents data corruption in the workflow.

In [4]:

#@title 3.Building Robust Components

# --- Hardening the call_llm Function ---
def call_llm_robust(system_prompt, user_content, retries=3, delay=5, model="gemini-2.0-flash"):
    """
    A robust helper function to call the Gemini API with retries.
    
    Args:
        system_prompt: System instructions to guide model behavior
        user_content: User's message/question
        retries: Number of retry attempts
        delay: Delay between retries in seconds
        model: Gemini model to use (default: gemini-2.0-flash)
        
    Returns:
        Response text or None if all retries fail
    """
    for i in range(retries):
        try:
            response = client.models.generate_content(
                model=model,
                contents=user_content,
                config=types.GenerateContentConfig(
                    system_instruction=system_prompt,
                    temperature=1.0,
                    top_p=0.95,
                    top_k=40,
                    max_output_tokens=8192,
                )
            )
            return response.text
        except Exception as e:
            print(f"API call failed on attempt {i+1}/{retries}. Error: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("All retries failed.")
                return None

# --- The MCP Validator ---
def validate_mcp_message(message):
    """A simple validator to check the structure of an MCP message."""
    required_keys = ["protocol_version", "sender", "content", "metadata"]
    if not isinstance(message, dict):
        print(f"MCP Validation Failed: Message is not a dictionary.")
        return False
    for key in required_keys:
        if key not in message:
            print(f"MCP Validation Failed: Missing key '{key}'")
            return False
    print(f"MCP message from {message['sender']} validated successfully.")
    return True

# 4.New: *Agent* Specialization Controls
This section expands our agent team from two to three. The original researcher_agent and writer_agent have been upgraded to use our new call_llm_robust function, making them more resilient. More importantly, we've introduced a new specialist: the validator_agent. This agent's sole purpose is to act as a fact-checker, comparing the writer's draft against the researcher's summary to ensure factual consistency, adding a vital quality control layer to our system.

In [5]:
#@title 4.Building the Agents: The Specialists

# --- Agent 1: The Researcher ---
def researcher_agent(mcp_input):
    """This agent takes a research topic, finds information, and returns a summary."""
    print("\n[Researcher Agent Activated]")
    simulated_database = {
        "mediterranean diet": "The Mediterranean diet is rich in fruits, vegetables, whole grains, olive oil, and fish. Studies show it is associated with a lower risk of heart disease, improved brain health, and a longer lifespan."
    }
    research_topic = mcp_input['content']
    research_result = simulated_database.get(research_topic.lower(), "No information found.")
    system_prompt = "You are a research analyst. Synthesize the provided information into 3-4 concise bullet points."
    summary = call_llm_robust(system_prompt, research_result)
    print(f"Research summary created for: '{research_topic}'")
    return create_mcp_message(
        sender="ResearcherAgent",
        content=summary,
        metadata={"source": "Simulated Internal DB"}
    )

# --- Agent 2: The Writer ---
def writer_agent(mcp_input):
    """This agent takes research findings and writes a short blog post."""
    print("\n[Writer Agent Activated]")
    research_summary = mcp_input['content']
    system_prompt = "You are a content writer. Take the following research points and write a short, appealing blog post (approx. 150 words) with a catchy title."
    blog_post = call_llm_robust(system_prompt, research_summary)
    print("Blog post drafted.")
    return create_mcp_message(
        sender="WriterAgent",
        content=blog_post,
        metadata={"word_count": len(blog_post.split())}
    )

# --- Agent 3: The Validator ---
def validator_agent(mcp_input):
    """This agent fact-checks a draft against a source summary."""
    print("\n[Validator Agent Activated]")
    source_summary = mcp_input['content']['summary']
    draft_post = mcp_input['content']['draft']
    system_prompt = """
    You are a meticulous fact-checker. Determine if the 'DRAFT' is factually consistent with the 'SOURCE SUMMARY'.
    - If all claims in the DRAFT are supported by the SOURCE, respond with only the word \"pass\".
    - If the DRAFT contains any information not in the SOURCE, respond with \"fail\" and a one-sentence explanation.
    """
    validation_context = f"SOURCE SUMMARY:\n{source_summary}\n\nDRAFT:\n{draft_post}"
    validation_result = call_llm_robust(system_prompt, validation_context)
    print(f"Validation complete. Result: {validation_result}")
    return create_mcp_message(
        sender="ValidatorAgent",
        content=validation_result
    )

# 5.New: Orchestrator Logic Controls
Here, the original simple orchestrator has been replaced with the far more intelligent final_orchestrator. This new version is no longer just a linear task manager. It now includes a validation and revision loop. After the writer_agent produces a draft, the orchestrator delegates to the validator_agent. If the validation fails, the orchestrator sends the draft back to the writer with the validator's feedback, creating a powerful, self-correcting system that mimics a real-world editorial process.

In [6]:
#@title 5.The Final Orchestrator with Validation Loop
def final_orchestrator(initial_goal):
    """Manages the full multi-agent workflow, including validation and revision."""
    print("="*50)
    print(f"[Orchestrator] Goal Received: '{initial_goal}'")
    print("="*50)

    # --- Step 1: Research ---
    print("\n[Orchestrator] Task 1: Research. Delegating to Researcher Agent.")
    research_topic = "Mediterranean Diet"
    mcp_to_researcher = create_mcp_message(sender="Orchestrator", content=research_topic)
    mcp_from_researcher = researcher_agent(mcp_to_researcher)

    if not validate_mcp_message(mcp_from_researcher) or not mcp_from_researcher['content']:
        print("Workflow failed due to invalid or empty message from Researcher.")
        return

    research_summary = mcp_from_researcher['content']
    print("\n[Orchestrator] Research complete.")

    # --- Step 2 & 3: Iterative Writing and Validation Loop ---
    final_output = "Could not produce a validated article."
    max_revisions = 2
    for i in range(max_revisions):
        print(f"\n[Orchestrator] Writing Attempt {i+1}/{max_revisions}")

        writer_context = research_summary
        if i > 0:
            writer_context += f"\n\nPlease revise the previous draft based on this feedback: {validation_result}"

        mcp_to_writer = create_mcp_message(sender="Orchestrator", content=writer_context)
        mcp_from_writer = writer_agent(mcp_to_writer)

        if not validate_mcp_message(mcp_from_writer) or not mcp_from_writer['content']:
            print("Aborting revision loop due to invalid message from Writer.")
            break
        draft_post = mcp_from_writer['content']

        # --- Validation Step ---
        print("\n[Orchestrator] Draft received. Delegating to Validator Agent.")
        validation_content = {"summary": research_summary, "draft": draft_post}
        mcp_to_validator = create_mcp_message(sender="Orchestrator", content=validation_content)
        mcp_from_validator = validator_agent(mcp_to_validator)

        if not validate_mcp_message(mcp_from_validator) or not mcp_from_validator['content']:
            print("Aborting revision loop due to invalid message from Validator.")
            break
        validation_result = mcp_from_validator['content']

        if "pass" in validation_result.lower():
            print("\n[Orchestrator] Validation PASSED. Finalizing content.")
            final_output = draft_post
            break
        else:
            print(f"\n[Orchestrator] Validation FAILED. Feedback: {validation_result}")
            if i < max_revisions - 1:
                print("Requesting revision.")
            else:
                print("Max revisions reached. Workflow failed.")

    # --- Step 4: Final Presentation ---
    print("\n" + "="*50)
    print("[Orchestrator] Workflow Complete. Final Output:")
    print("="*50)
    print(final_output)


# 6.New:Final System Execution Controls
This final code block now runs the complete, upgraded system. Instead of calling the simple prototype, it executes the final_orchestrator. This allows the reader to see the entire, robust workflow in action, including the new validation and revision steps, demonstrating the full power of the resilient, multi-agent architecture we have built.

In [7]:
#@title 6.Run the Final, Robust System
user_goal = "Create a blog post about the benefits of the Mediterranean diet."
final_orchestrator(user_goal)

[Orchestrator] Goal Received: 'Create a blog post about the benefits of the Mediterranean diet.'

[Orchestrator] Task 1: Research. Delegating to Researcher Agent.

[Researcher Agent Activated]


Research summary created for: 'Mediterranean Diet'
MCP message from ResearcherAgent validated successfully.

[Orchestrator] Research complete.

[Orchestrator] Writing Attempt 1/2

[Writer Agent Activated]
Blog post drafted.
MCP message from WriterAgent validated successfully.

[Orchestrator] Draft received. Delegating to Validator Agent.

[Validator Agent Activated]
Validation complete. Result: pass

MCP message from ValidatorAgent validated successfully.

[Orchestrator] Validation PASSED. Finalizing content.

[Orchestrator] Workflow Complete. Final Output:
## Unlock a Healthier You: Discover the Mediterranean Diet

Want to eat your way to a healthier, happier life? Look no further than the Mediterranean diet! This isn't just a trendy fad; it's a time-tested approach to eating that emphasizes delicious, wholesome foods.

Imagine plates bursting with colorful fruits and vegetables, drizzled with heart-healthy olive oil, and featuring plenty of whole grains and lean fish. That's the esse